In [ ]:
import os

def check_subdirectories(directory):
    all_subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    flag = True
    for subdir in all_subdirs:
        subdir_path = os.path.join(directory, subdir)
        files = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]
        
        if len(files) != 4:
            print(f"Subdirectory '{subdir}' does not contain exactly 4 files. It contains {len(files)} files.")
            # return False
            flag = False
    
    print("All first-level subdirectories contain exactly 4 files.")
    return flag

# Example usage
directory = "/work/jiaqi/Retrieval/data/PDBbind_v2020/v2020-other-PL"
check_subdirectories(directory)

In [ ]:
import os

def count_first_level_subdirectories(directory):
    subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return len(subdirs)

# Example usage
directory = "../data/PDBBind_processed"
num_subdirectories = count_first_level_subdirectories(directory)
print(f"Number of first-level subdirectories: {num_subdirectories}")


In [ ]:
import pandas as pd

data = pd.read_csv('../data/idmapping_2024_08_28.tsv', sep='\t')
entries = data['From'].tolist()
with open('../data/PDBids.txt') as f:
    pdbids = f.read().split('\n')
k = 0
for pdb in pdbids:
    if pdb not in entries:
        print(pdb)
        k += 1
print(k)

In [ ]:
import os
import re

def extract_chain_ids(selection_string):
    # Use a regular expression to find all chain IDs
    chain_ids = re.findall(r"chain\s+([A-Za-z])", selection_string)
    return chain_ids

with open('../data/PDBids.txt') as f:
    pdbids = f.read().split('\n')
print(f'Number of PDB IDs: {len(pdbids)}')
pdb_chain_ids = []
for pdb in pdbids:
    with open(f'../data/PDBBind_processed/{pdb}/{pdb}_protein_processed.pdb') as f:
        lines = f.readlines()
    remark = lines[0]
    chain_ids = extract_chain_ids(remark)
    for chain_id in chain_ids:
        pdb_chain_ids.append(f'{pdb}-{chain_id}')
print(f'Number of PDB chain IDs: {len(pdb_chain_ids)}')
with open('../data/PDB_chain_ids.txt', 'w') as f:
    for pdb_chain_id in pdb_chain_ids:
        f.write(f'{pdb_chain_id}\n')


In [ ]:
from rdkit import Chem

def parse_sdf(file_path):
    suppl = Chem.SDMolSupplier(file_path)
    
    for mol in suppl:
        if mol is None:  # skip empty molecules
            continue
            
        # Get the molecule name (assumes the name is stored in the '_Name' property)
        name = mol.GetProp('_Name') if mol.HasProp('_Name') else 'Unnamed'
        
        # Get the SMILES representation
        smiles = Chem.MolToSmiles(mol)
        
        print(f"Molecule Name: {name}")
        print(f"SMILES: {smiles}")
        print()

# Example usage
file_path = "../data/PDBBind_processed/185l/185l_ligand.sdf"
# parse_sdf(file_path)
mol = Chem.SDMolSupplier(file_path)


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolfiles

def parse_mol2_with_rdkit(file_path):
    mol = rdmolfiles.MolFromMol2File(file_path)
    
    if mol is not None:
        # Get the SMILES representation
        smiles = Chem.MolToSmiles(mol)
        
        print(f"SMILES: {smiles}")
        print()
    else:
        print("Could not parse the MOL2 file.")

# Example usage
file_path = "../data/PDBBind_processed/185l/185l_ligand.mol2"
parse_mol2_with_rdkit(file_path)


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolfiles

def parse_sdf(file_path):
    # Assuming the file contains only one molecule
    suppl = Chem.SDMolSupplier(file_path)
    mol = next(suppl)  # Get the first molecule
    return mol

def parse_mol2(file_path):
    mol = rdmolfiles.MolFromMol2File(file_path)
    return mol

def compare_molecules(mol1, mol2):
    if mol1 is None or mol2 is None:
        return False
    
    # Generate canonical SMILES
    smiles1 = Chem.MolToSmiles(mol1, canonical=True)
    smiles2 = Chem.MolToSmiles(mol2, canonical=True)
    print(smiles1)
    print(smiles2)
    
    return smiles1 == smiles2

# Example usage
sdf_file_path = "../data/PDBBind_processed/185l/185l_ligand.sdf"
mol2_file_path = "../data/PDBBind_processed/185l/185l_ligand.mol2"

mol_from_sdf = parse_sdf(sdf_file_path)
mol_from_mol2 = parse_mol2(mol2_file_path)

if compare_molecules(mol_from_sdf, mol_from_mol2):
    print("The molecules are the same.")
else:
    print("The molecules are different.")


In [ ]:
from rdkit import Chem
import pandas as pd

def parse_sdf(file_path):
    # Assuming the file contains only one molecule
    try:
        suppl = Chem.SDMolSupplier(file_path)
        mol = next(suppl)  # Get the first molecule
        smiles = Chem.MolToSmiles(mol, canonical=True)
        return mol, smiles
    except:
        print(f"Could not parse the SDF file {file_path}.")
        return None, ''

def parse_mol2_with_rdkit(file_path):
    mol = rdmolfiles.MolFromMol2File(file_path)
    
    if mol is not None:
        # Get the SMILES representation
        smiles = Chem.MolToSmiles(mol)
        return mol, smiles
    else:
        print("Could not parse the MOL2 file.")

with open('../data/PDBids.txt') as f:
    pdbids = f.read().split('\n')
print(f'Number of PDB IDs: {len(pdbids)}')
smiles_list = []
for pdb in pdbids:
    # print(pdb)
    sdf_file_path = f'../data/PDBBind_processed/{pdb}/{pdb}_ligand.sdf'
    mol, smiles = parse_sdf(sdf_file_path)
    smiles_list.append(smiles)
print(f'Number of SMILES: {len(smiles_list)}')
print(f'Number of unique SMILES: {len(set(smiles_list))}')
df = pd.DataFrame({'PDB ID': pdbids, 'SMILES': smiles_list})
df.to_csv('../data/PDB_smiles.csv', index=False)

In [ ]:
from rdkit import Chem
import warnings
from rdkit.Chem import MolFromPDBFile, AllChem, GetPeriodicTable, rdDistGeom

def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=False):
    """Load a molecule from a file of format ``.mol2`` or ``.sdf`` or ``.pdbqt`` or ``.pdb``.

    Parameters
    ----------
    molecule_file : str
        Path to file for storing a molecule, which can be of format ``.mol2`` or ``.sdf``
        or ``.pdbqt`` or ``.pdb``.
    sanitize : bool
        Whether sanitization is performed in initializing RDKit molecule instances. See
        https://www.rdkit.org/docs/RDKit_Book.html for details of the sanitization.
        Default to False.
    calc_charges : bool
        Whether to add Gasteiger charges via RDKit. Setting this to be True will enforce
        ``sanitize`` to be True. Default to False.
    remove_hs : bool
        Whether to remove hydrogens via RDKit. Note that removing hydrogens can be quite
        slow for large molecules. Default to False.
    use_conformation : bool
        Whether we need to extract molecular conformation from proteins and ligands.
        Default to True.

    Returns
    -------
    mol : rdkit.Chem.rdchem.Mol
        RDKit molecule instance for the loaded molecule.
    coordinates : np.ndarray of shape (N, 3) or None
        The 3D coordinates of atoms in the molecule. N for the number of atoms in
        the molecule. None will be returned if ``use_conformation`` is False or
        we failed to get conformation information.
    """
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        return ValueError('Expect the format of the molecule_file to be '
                          'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)
    except:
        return None

    return mol

file_path = "../data/PDBBind_processed/185l/185l_ligand.sdf"
mol = read_molecule(file_path)
smiles = Chem.MolToSmiles(mol)
print(mol)
print(smiles)

In [ ]:
import pandas as pd

df = pd.read_csv('../data/PDB_smiles.csv')
df = df.replace(pd.NA, '')
smiles = df['SMILES'].tolist()
print(f'Number of SMILES: {len(smiles)}')
print(f'Number of unique SMILES: {len(set(smiles))}')
empty_df = df[df['SMILES'] == '']
empty_df

In [ ]:
import sys
sys.path.append('..')
from utils.process_molecules import read_molecule
from rdkit import Chem
import pandas as pd

with open('../data/PDBids.txt') as f:
    pdbids = f.read().split('\n')
print(f'Number of PDB IDs: {len(pdbids)}')
smiles_list = []
for pdb in pdbids:
    # print(pdb)
    sdf_file_path = f'../data/PDBBind_processed/{pdb}/{pdb}_ligand.sdf'
    mol = read_molecule(sdf_file_path, sanitize=False, remove_hs=True)
    smiles = Chem.MolToSmiles(mol, canonical=True)
    smiles_list.append(smiles)
print(f'Number of SMILES: {len(smiles_list)}')
print(f'Number of unique SMILES: {len(set(smiles_list))}')
df = pd.DataFrame({'PDB ID': pdbids, 'SMILES': smiles_list})
df.to_csv('../data/PDB_smiles.csv', index=False)

In [5]:
import torch
import pandas as pd

emb_dir = '/work/jiaqi/ProtRepr/data/esm1b_emb_price'
df = pd.read_csv('/work/jiaqi/ProtRepr/data/ec_new/price.csv')
entries = df['Entry'].tolist()
ec_numbers = df['EC number'].tolist()
ec_numbers = [ec.split(';') for ec in ec_numbers]
data = {}
n = len(entries)
for i in range(n):
    entry = entries[i]
    ec_number = ec_numbers[i]
    emb_path = f'{emb_dir}/{entry}.pt'
    emb = torch.load(emb_path)['mean_representations'][33]
    data[entry] = {'embedding': emb, 'ec': ec_number}
print(len(data))
print(data)
torch.save(data, '../data/ec/price.pt')

149
{'WP_063460136': {'embedding': tensor([-0.0602,  0.0113,  0.1865,  ...,  0.0939, -0.0376,  0.0231]), 'ec': ['5.3.1.7']}, 'WP_063462980': {'embedding': tensor([-0.0081,  0.2320, -0.0246,  ...,  0.0220, -0.0463,  0.1308]), 'ec': ['4.2.1.43']}, 'WP_063462990': {'embedding': tensor([-0.0185,  0.0809, -0.0025,  ..., -0.0075, -0.0291,  0.0270]), 'ec': ['1.1.1.48']}, 'WP_041412631': {'embedding': tensor([ 0.1510,  0.2038,  0.0316,  ..., -0.0337, -0.1527, -0.0112]), 'ec': ['4.2.1.25']}, 'WP_011717048': {'embedding': tensor([ 0.1011,  0.1449, -0.1279,  ..., -0.0622, -0.2993,  0.2904]), 'ec': ['5.1.3.3']}, 'WP_011717064': {'embedding': tensor([ 0.1233,  0.2573, -0.0023,  ...,  0.0191, -0.0820,  0.2154]), 'ec': ['3.1.1.15']}, 'WP_011717945': {'embedding': tensor([ 0.0830,  0.1686,  0.0215,  ..., -0.0622, -0.0645,  0.1001]), 'ec': ['2.7.1.59']}, 'WP_014239822': {'embedding': tensor([ 0.0601,  0.2106,  0.0102,  ..., -0.1288, -0.1465,  0.1254]), 'ec': ['1.3.8.7']}, 'WP_012431492': {'embedding': 

In [2]:
import pandas as pd

with open('../data/PDBbind_v2020/PDBbind_v2020_plain_text_index/index/INDEX_general_PL_data.2020') as f:
    lines = f.readlines()
ligands = []
for line in lines:
    if not line.startswith('#'):
        ligands.append(line.split()[-1][1:-1])
print(ligands[:10])
print(f'unique ligands: {len(set(ligands))}')

['NLG', 'SFX', '1P3', 'GAB&PMP', 'AZM', 'MFU', '2SU', 'MET', 'M6D', 'CIT']
unique ligands: 12927


In [3]:
with open('../data/PDBbind_v2020/PDBbind_v2020_plain_text_index/index/INDEX_general_PL_name.2020') as f:
    lines = f.readlines()
uniprot_ids = []
for line in lines:
    if not line.startswith('#'):
        uniprot_ids.append(line.split()[2])
print(uniprot_ids[:10])
print(f'unique uniprot IDs: {len(set(uniprot_ids))}')

['P29994', 'P29994', 'P11881', 'O75643', 'O75643', 'O75643', 'Q9Z9H6', 'Q5SHR6', 'Q5SHR6', 'P13569']
unique uniprot IDs: 3890


In [5]:
import json
import torch

with open('../data/ec/ec_list_by2022-05-25.json') as f:
    labels = json.load(f)
data = torch.load('../data/ec/price.pt')
for k, v in data.items():
    for ec in v['ec']:
        if ec not in labels:
            print(ec)
            labels.append(ec)
print(len(labels), len(set(labels)))
with open('../data/ec/ec_list_by2022-05-25_price.json', 'w') as f:
    json.dump(labels, f)

3.5.1.30
1.13.12.2
1.1.99.13
5341 5341
